# Engine power vs emissions

Click {fa}`rocket` --> {guilabel}`Live Code` to activate live coding on this page!




## Problem
In this problem we're trying to optimize a diesel engine for maximum power and minimum CO<sub>2</sub> emissions. Depending on the optimal engine speed, the power and CO<sub>2</sub> emissions change.

The following data is available:

| Rotations per minute  | CO<sub>2</sub> Emissions | Power   |
|------|---------------|---------|
| 800  | 708.       | 161.141 |
| 1000 | 696.889       | 263.243 |
| 1200 | 688.247       | 330.51 |
| 1400 | 682.897       | 381.561 |
| 1700 | 684.955       | 391.17 |
| 1800 | 697.3       | 370. |

This data is interpolated to obtain a continuous relation:

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']

RPM = np.array([800, 1000, 1200, 1400, 1700, 1800])
CO2 = np.array([708, 696.889, 688.247, 682.897, 684.955, 697.3 ])
POW = np.array([161.141, 263.243, 330.51 , 381.561, 391.17, 370 ])

def CO2func(s):
    return sp.interpolate.pchip_interpolate(RPM,CO2,s)

def POWfunc(s):
    return sp.interpolate.pchip_interpolate(RPM,POW,s)

RPM_continuous = np.linspace(800,1800,100)
plt.figure()
plt.subplot(2,1,1)
plt.plot(RPM, CO2, 'x', label='Original Data')
plt.plot(RPM_continuous, CO2func(RPM_continuous), label='Interpolated Data')
plt.xlabel('RPM')
plt.ylabel('CO$_2$ (g/kWh)')
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.tight_layout()
plt.legend()

plt.subplot(2,1,2)
plt.plot(RPM, POW, 'x', label='Original Data')
plt.plot(RPM_continuous, POWfunc(RPM_continuous), label='Interpolated Data')
plt.xlabel('RPM')
plt.ylabel('Power (kW)')
ax = plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
plt.tight_layout()
plt.legend();


```{figure} ./figures/CO2_power.svg
:name: RPM-CO2_power
:width: 600px
Interpolation of CO<sub>2</sub> emissions and Power
```

## Model

Let's define our model in three different ways, as defined in {eq}`multi_objective_optimization_weighted`, {eq}`multi_objective_optimization_goal` and {eq}`multi_objective_optimization_pareto`.

We'll define the model as follows:
- Design variables: scalar value representing the rotations per minute ($\text{RPM}$)
- Objective function: weighted sum of the power and CO<sub>2</sub> emissions, max of difference with goals or pareto front
- Bounds: only interpolation so between $800$ and $1800$ $\text{RPM}$

### Design variables
Let's start with our design variables. In this case a logical choice could be the width, height and depth of our block

```{math}
:label: moo_x
x= \text{rotations per minute}
```

### Objective function

Let's explore all three possible objective functions:

#### Weighted objective function
We can define the weighted objective function as defined by {eq}`multi_objective_optimization_weighted` with some predefined weights. For example a weight of $\cfrac{1}{3}$ for the power value and $\cfrac{2}{3}$ for the CO<sub>2</sub> emissions.

```{math}
:label: multi_objective_optimization_weighted_example
 \mathop {\min }\limits_x \left( -\cfrac{1}{3} \cdot f_{\text{power}}\left( x \right) + \cfrac{2}{3} \cdot f_{\text{CO}_2\text{ emissions}}\left( x \right) \right)
```

Please note that we need a minus for the power objective because it's an maximization objective and we now apply the weights to the non-normalized functions while they have different units. Therefore, it might be wise to apply normalization as defined by {eq}`normalizing_f`.

#### Goal attainment
If we define two fails for the power and CO<sub>2</sub> emissions, we can apply goal attainment. Those goals could be $460 \text{ kW}$ for the generated power and $640 \text{ g/kWh}$ for the CO<sub>2</sub> emissions. Now the objective function can as in {eq}`multi_objective_optimization_goal` as:

```{math}
:label: multi_objective_optimization_goal_example
 \mathop {\min }\limits_x \left( 460 - \max \left( f_{\text{power}}\left( x \right), f_{\text{CO}_2\text{ emissions}} \left( x \right) - 640 \right) \right)
```

#### Pareto front
Finally, we can find the pareto front by defining the problem as in {eq}`multi_objective_optimization_pareto`:

```{math}
:label: multi_objective_optimization_pareto_example
 \mathop {\min }\limits_x \left( {{\delta }_{i}} \cdot f_{\text{power},\text{normalized}}\left( x \right) +  \delta_j \cdot f_{\text{CO}_2\text{ emissions},\text{normalized}}\left( x \right) \right)
```

For normalization, we could take the value from our plot, but for more real-life complex problems the dimensions are higher and you cannot find the maximum and minimum of each single objective function. Therefore, let's estimate the maxima and minima as:

|  | CO<sub>2</sub> Emissions | Power   |
|------|---------------|---------|
| Minimum value  | 680       | 150 |
| Maximum value | 710       | 400 |

This gives:

```{math}
:label: normalizing_f_example
\begin{matrix}
  {{f}_{\text{power, normalized}}}\left( x \right)=\cfrac{f_{\text{power}}\left( x \right)- 150}{400-150} \\ 
  {{f}_{\text{CO}_2\text{ emissions},\text{normalized}}}\left( x \right)=\cfrac{f_{\text{CO}_2\text{ emissions}}\left( x \right)- 680}{710-680} \\ 
\end{matrix}
```

To find the full pareto front this optimization model has to be solved for a large set of $\delta_i$ and $\delta_j$

### Bounds

Let's limit our solution to within our interpolation domain. Therefore, the bound can be defined as:

```{math}
:label: bounds_moo
800<x < 1800
```

### Find best solution manually

:::{card} Test yourself
Try and adjust the values for $x_1$, $x_2$ and $x_3$. Can you find the optimal solution?
<iframe src="../_static/block.html" style="overflow:hidden;height:350;width:100%" height="350" width="100%"> frameborder="0"></iframe>
:::

In [17]:
from ipywidgets import widgets, interact

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']

RPM = np.array([800, 1000, 1200, 1400, 1700, 1800])
CO2 = np.array([708, 696.889, 688.247, 682.897, 684.955, 697.3 ])
POW = np.array([161.141, 263.243, 330.51 , 381.561, 391.17, 370 ])

def weighted_obj(s):
    delta_p = 1/3
    delta_c = 1 - delta_p
    return -delta_p * POWfunc(s) + delta_c * CO2func(s)

Pt = 460
Ct = 640
def goal_attainment(s):
    return max(Pt - POWfunc(s),CO2func(s)-Ct)

def CO2func(s):
    return sp.interpolate.pchip_interpolate(RPM,CO2,s)

def POWfunc(s):
    return sp.interpolate.pchip_interpolate(RPM,POW,s)

def func(x,pareto_front=True):
    fig, ax = plt.subplots(1, 1)
    ax.clear()
    ax.plot(CO2func(x),POWfunc(x),'x')
    if pareto_front:
        ax.plot(CO2func(RPM_continuous),POWfunc(RPM_continuous))
    ax.set_xlim([680,710])
    ax.set_ylim([150,400])
    ax.invert_yaxis()
    ax.set_xlabel('CO$_2$ (g/kWh)')
    ax.set_ylabel('Power functions (kW)')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.set_title('CO$_2$ vs Power')
    plt.draw()
    weighted_obj_val = weighted_obj(x)
    goal_attainment_val = goal_attainment(x)
    print('Weighted 🏋️ Objective:',weighted_obj_val)
    print('Goal 🥅 Attainment:',goal_attainment_val)

interact(func, x = widgets.IntSlider(min=800, max=1800, value=1000, step=1, description="RPM"), pareto_front = widgets.Checkbox(value=False, description="All possible RPM"));

interactive(children=(IntSlider(value=1000, description='RPM', max=1800, min=800), Checkbox(value=False, descr…

## Method

Now let's solve this problem using an optimization method.

### Import libraries

In [ ]:
import scipy as sp
import numpy as np

In [ ]:
import scipy as sp 
import numpy as np

### Define variables
As before, we don't need to specify our variable $x$ itself as defined in {eq}`nonlinear_constrained_optimization_x`. However, this optimization method requires an initial guess. An arbitrary value is chosen here:

In [ ]:
x0 = np.array([5,0,1])

### Define objective function

The objective function was defined in {eq}`nonlinear_constrained_optimization_f`, which gives:

In [ ]:
def func(x):
    vol = x[0]*x[1]*x[2]
    return vol

### Define constrain functions

The constraint functions were defined in {eq}`nonlinear_constrained_optimization_g`. We had no equality constraints. Unlike before with {ref}`the method to solve linear constrained problem <linear_constraint_function_method>`, we need an object which defines the upper and lower bounds. As this problem has only an upper bound of $0$, the lower bound is set to $\infty$ which is `np.inf` in python. Note that a single constraint object can include multiple constraints.

In [ ]:
def nonlinconfun(x):
    c1 = 0.8 - x[0]*x[1]
    c2 = 0.8 - x[0]*x[2]
    c3 = 0.8 - x[1]*x[2]
    c4 = 3000 - 2500 * x[0] * x[1] * x[2]
    return np.array([c1,c2,c3,c4])

cons = sp.optimize.NonlinearConstraint(nonlinconfun, np.array([-np.inf,-np.inf,-np.inf,-np.inf]), np.array([0,0,0,0]))

### Define bounds
The bounds were defined in {eq}`bounds_nonlinear` and result in:

In [ ]:
bounds = [[0, None],
          [0, None],
          [0, None]]

### Solve the problem

Now let's solve the problem. The `cons` object can be added directly, in the case of equality constraints as well you can define a list of constrainer objects as an input.

In [ ]:
result = sp.optimize.minimize(fun = func,x0 = x0,bounds = bounds,constraints=cons)
print(result)

## Exercise

:::{card} Test yourself
Click {fa}`rocket` --> {guilabel}`Live Code` to activate live coding on this page.
<iframe src="https://tudelft.h5p.com/content/1292254715005702697/embed" aria-label="apply" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>
:::

## Questions, discussions and comments
<script src="https://utteranc.es/client.js"
        repo="TeachBooks/engineering-systems-optimization"
        issue-term="title"
        theme="github-light"
        crossorigin="anonymous"
        async>
</script>